<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Tutorials/Structured%20Data/Feature%20Columns/Notebook1_Classify_structured_data_with_feature_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Insatll TensorFlow 2.x

In [0]:
# !pip install --upgrade tensorflow

     |████████████████████████████████| 86.3MB 32.3MB/s 
     |████████████████████████████████| 450kB 40.3MB/s 
     |████████████████████████████████| 3.8MB 27.2MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0rc3
    Uninstalling tensorflow-1.15.0rc3:
      Successfully uninstalled tensorflow-1.15.0rc3


### Import TensorFlow and other Libraries

In [0]:
%%time
%reload_ext autoreload
%autoreload 2
%reload_ext tensorboard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from tensorflow import feature_column
from tensorflow.keras import layers
keras = tf.keras

print(f"TensorFlow version {tf.__version__}, Keras version {keras.__version__}")
if tf.config.experimental.list_physical_devices("GPU"):
    print("GPU is avaialble.")
print(f"TensorFlow is executing eagerly: {tf.executing_eagerly()}")

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

TensorFlow version 2.0.0, Keras version 2.2.4-tf
TensorFlow is executing eagerly: True
CPU times: user 46.1 ms, sys: 12 ms, total: 58.2 ms
Wall time: 58.3 ms


### Use Pandas to create a DataFrame

In [0]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [0]:
dataframe.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

### Split the data into train and validation sets

In [0]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Validation shape: {val.shape}")

Train shape: (193, 14)
Test shape: (61, 14)
Validation shape: (49, 14)


### Create an input pipeline with tf.data

In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=64):
  dataframe = dataframe.copy()
  labels = dataframe.pop("target")
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 64

train_ds = df_to_dataset(train, batch_size=batch_size)

val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [0]:
print(f"Training dataset: {train_ds.element_spec}")

print(f"Test dataset: {test_ds.element_spec}")

print(f"Validation dataset: {val_ds.element_spec}")

Training dataset: ({'age': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'sex': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'cp': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'trestbps': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'chol': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'fbs': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'restecg': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'thalach': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'exang': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'oldpeak': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'slope': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'ca': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'thal': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Test dataset: ({'age': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'sex': TensorSpec(shape=(None,)

### Understanding the input pipeline

In [0]:
for feature_batch, label_batch in train_ds.take(1):
  print(f"Label Batch: {label_batch}")
  print(f"A batch of feature values: {feature_batch.values()}")
  print(f"All features: {feature_batch.keys()}")

Label Batch: [0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1]
A batch of feature values: dict_values([<tf.Tensor: id=309, shape=(64,), dtype=int32, numpy=
array([57, 41, 71, 58, 44, 55, 67, 46, 63, 59, 52, 39, 56, 43, 70, 56, 42,
       46, 59, 60, 56, 63, 42, 63, 67, 37, 64, 48, 56, 41, 41, 56, 52, 52,
       77, 58, 60, 66, 45, 50, 65, 54, 62, 58, 62, 67, 54, 57, 59, 43, 70,
       42, 58, 54, 43, 54, 54, 65, 68, 45, 52, 69, 62, 60], dtype=int32)>, <tf.Tensor: id=317, shape=(64,), dtype=int32, numpy=
array([0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
      dtype=int32)>, <tf.Tensor: id=312, shape=(64,), dtype=int32, numpy=
array([1, 3, 3, 4, 3, 4, 4, 4, 4, 1, 4, 3, 1, 4, 2, 4, 2, 2, 4, 4, 4, 4,
       3, 3, 3, 3, 3, 2, 2, 2, 3, 2, 1, 4, 4, 4, 

### Demonstrate several types of feature columns

In [0]:
example_batch = next(iter(train_ds))[0]

In [0]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [0]:
age = feature_column.numeric_column("age")
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[58.]
 [63.]
 [64.]
 [49.]
 [62.]
 [68.]
 [40.]
 [67.]
 [42.]
 [56.]
 [65.]
 [29.]
 [34.]
 [63.]
 [54.]
 [41.]
 [62.]
 [43.]
 [39.]
 [64.]
 [65.]
 [53.]
 [67.]
 [42.]
 [62.]
 [55.]
 [52.]
 [47.]
 [59.]
 [50.]
 [51.]
 [60.]
 [67.]
 [44.]
 [60.]
 [40.]
 [63.]
 [46.]
 [61.]
 [46.]
 [41.]
 [64.]
 [77.]
 [52.]
 [58.]
 [62.]
 [53.]
 [70.]
 [46.]
 [35.]
 [60.]
 [71.]
 [53.]
 [62.]
 [57.]
 [41.]
 [54.]
 [57.]
 [55.]
 [44.]
 [46.]
 [62.]
 [63.]
 [43.]]


In [0]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 40, 45, 50, 55, 65, 70, 85])

In [0]:
demo(age_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0.

In [0]:
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal', ['fixed', 'normal', 'reversible']
)
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 

In [0]:
thal_embedding = feature_column.embedding_column(thal, dimension=10)
demo(thal_embedding)

In [0]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
    'thal', hash_bucket_size=1000
)
demo()
